The idea with this nootebooks is to try a LOCI alike algorithm to fix caravan precipitation. The correcion would take the next steps:
* Select the days when it actually rains on CAMELS
* Continue with the rest of the correction process

In [1]:
from pathlib import Path
import xarray as xr
import numpy as np
from scipy.stats import gamma
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd

In [6]:
# NOTE: Both datasets range froom 1989-01-01 to 2019-12-31 so no need to slice by date

In [2]:
# ---------------- Paths ----------------
data_caravan = Path("../preparing_data/filtered_data/time_series")
data_gauge  = Path("../preparing_data/cleaned_filtered_data_gauge_precip/time_series")
output_dir   = Path("./new_correction_approach/time_series")

output_dir.mkdir(parents=True, exist_ok=True)

# # ---------------- Dates ----------------
# start_date = pd.Timestamp("1989-10-01")
# end_date   = pd.Timestamp("2008-09-30")

eps = 1e-6
time_dim = "date"

# ---------------- Load basin list ----------------
with open("filtered_uy_basins.txt") as f:
    basins = [line.strip() for line in f if line.strip()]

print(f"Processing {len(basins)} basins")

# ---------------- Loop over basins ----------------
for basin in basins:
    print(f"\n▶ Processing basin {basin}")

    # -------- Caravan file --------
    caravan_file = data_caravan / f"{basin}.nc"
    if not caravan_file.exists():
        print(f"  ⚠️ Caravan file not found, skipping")
        continue

    caravan = xr.open_dataset(caravan_file)

    # -------- Caravan file --------
    gauge_file = data_gauge / f"{basin}.nc"
    if not gauge_file.exists():
        print(f"  ⚠️ Gauge file not found, skipping")
        continue

    gauge = xr.open_dataset(gauge_file)

    # # -------- Time slice --------
    # caravan = caravan.sel(date=slice(start_date, end_date))
    # gauge = gauge.sel(date=slice(start_date, end_date))

    # -------- Select variables --------
    vars_keep = [
        "prcp_mm_day",
        "tmax_C",
        "tmin_C",
        "srad_W_m2",
        "QObs_mm_d",
    ]

    # -------- Wet-day mask from observations --------
    mask = gauge["prcp_mm_day"] > 0

    caravan_masked = caravan.copy()

    caravan_masked["prcp_mm_day"] = (
        caravan_masked["prcp_mm_day"].where(mask, 0)
    )

    # -------- Define obs / era precipitation --------
    pr_era = caravan_masked["prcp_mm_day"]

    pr_obs = gauge["prcp_mm_day"]

    # -------- Add month coordinate --------
    pr_obs = pr_obs.assign_coords(month=pr_obs[time_dim].dt.month)
    pr_era = pr_era.assign_coords(month=pr_era[time_dim].dt.month)

    # -------- Monthly means --------
    obs_monthly_mean = pr_obs.groupby("month").mean(time_dim)
    era_monthly_mean = pr_era.groupby("month").mean(time_dim)

    # -------- Scaling factor --------
    scaling_factor = obs_monthly_mean / (era_monthly_mean + eps)

    # -------- Apply LS correction --------
    pr_era_ls = pr_era.groupby(f"{time_dim}.month") * scaling_factor

    # -------- Store back into dataset --------
    caravan["prcp_mm_day"] = pr_era_ls.drop_vars(
        "month", errors="ignore"
    )

    # -------- Save --------
    output_path = output_dir / f"{basin}.nc"
    caravan.to_netcdf(output_path)

    caravan.close()
    gauge.close()

    print(f"  ✅ Saved to {output_path}")


Processing 11 basins

▶ Processing basin CAMELS_UY_2
  ✅ Saved to new_correction_approach/time_series/CAMELS_UY_2.nc

▶ Processing basin CAMELS_UY_3
  ✅ Saved to new_correction_approach/time_series/CAMELS_UY_3.nc

▶ Processing basin CAMELS_UY_5
  ✅ Saved to new_correction_approach/time_series/CAMELS_UY_5.nc

▶ Processing basin CAMELS_UY_6
  ✅ Saved to new_correction_approach/time_series/CAMELS_UY_6.nc

▶ Processing basin CAMELS_UY_7
  ✅ Saved to new_correction_approach/time_series/CAMELS_UY_7.nc

▶ Processing basin CAMELS_UY_8
  ✅ Saved to new_correction_approach/time_series/CAMELS_UY_8.nc

▶ Processing basin CAMELS_UY_9
  ✅ Saved to new_correction_approach/time_series/CAMELS_UY_9.nc

▶ Processing basin CAMELS_UY_10
  ✅ Saved to new_correction_approach/time_series/CAMELS_UY_10.nc

▶ Processing basin CAMELS_UY_11
  ✅ Saved to new_correction_approach/time_series/CAMELS_UY_11.nc

▶ Processing basin CAMELS_UY_15
  ✅ Saved to new_correction_approach/time_series/CAMELS_UY_15.nc

▶ Processing

In [3]:
ds = xr.open_dataset(data_caravan / "CAMELS_UY_2.nc")
# ds = ds.sel(date=slice(start_date, end_date))

n_zeros = (ds["prcp_mm_day"] == 0).sum().item()
total_precip = ds["prcp_mm_day"].sum(dim="date").values

print(n_zeros)
print('Total acumulated precip:',total_precip)

3932
Total acumulated precip: 45184.78


In [4]:
ds_gauge = xr.open_dataset(data_gauge / "CAMELS_UY_2.nc")
# ds_gauge = ds_gauge.sel(date=slice(start_date, end_date))

n_zeros_gauge = (ds_gauge["prcp_mm_day"] == 0).sum().item()
total_precip_gauge = ds_gauge["prcp_mm_day"].sum(dim="date").values

print(n_zeros_gauge)
print('Total acumulated precip:',total_precip_gauge)

7832
Total acumulated precip: 49271.7


In [5]:
ds_corrected = xr.open_dataset(output_dir / "CAMELS_UY_2.nc")
# ds_corrected = ds_corrected.sel(date=slice(start_date, end_date))

n_zeros_corrected = (ds_corrected["prcp_mm_day"] == 0).sum().item()
total_precip_corrected = ds_corrected["prcp_mm_day"].sum(dim="date").values

print(n_zeros_corrected)
print('Total acumulated precip:',total_precip_corrected)

7948
Total acumulated precip: 49271.688
